https://blog.csdn.net/weixin_39915649/article/details/135213748

In [3]:
import time
import concurrent.futures
 
 
Stime = time.perf_counter()
tasks = []
sleepTimes = [0.1, 0.2, 0.1, 0.5, 0.7, 0.9, 0.5,
              0.4, 1.5, 1.3, 1.0, 0.3, 0.7, 0.6, 0.3, 0.8]
print(f"Total time of sleep: {sum(sleepTimes)} for {len(sleepTimes)} tasks")
 
def my_awesome_function(sleepTime=0.1):
    time.sleep(sleepTime)
    return f"Sleep time {sleepTime}"
 
all_results = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    tasks = [executor.submit(my_awesome_function, sleep)
             for sleep in sleepTimes]
 
    for ff in concurrent.futures.as_completed(tasks):
        all_results.append(ff.result())
 
 
print(f"Finished in {time.perf_counter()-Stime:.2f}")

Total time of sleep: 9.9 for 16 tasks
Finished in 1.66


In [5]:
import time
import ray
 
import concurrent.futures
 
 
Stime = time.perf_counter()
tasks = []
sleepTimes = [0.1, 0.2, 0.1, 0.5, 0.7, 0.9, 0.5,
              0.4, 1.5, 1.3, 1.0, 0.3, 0.7, 0.6, 0.3, 0.8]
print(f"Total time of sleep: {sum(sleepTimes)} for {len(sleepTimes)} tasks")
 
# 初始化Ray。
ray.shutdown()
ray.init()
 
@ray.remote #convert to a function that can be executed remotely and asynchronously
def my_awesome_function(sleepTime=0.1):
    time.sleep(sleepTime)
    return f"Sleep time {sleepTime}"
 
tasks = []
for sleep in sleepTimes:
    tasks.append(my_awesome_function.remote(sleep))
 
all_results = ray.get(tasks)
print(f"Finished in {time.perf_counter()-Stime:.2f}")

Total time of sleep: 9.9 for 16 tasks


2024-04-06 11:26:31,620	INFO worker.py:1538 -- Started a local Ray instance.


Finished in 7.82


In [7]:
all_results

['Sleep time 0.1',
 'Sleep time 0.2',
 'Sleep time 0.1',
 'Sleep time 0.5',
 'Sleep time 0.7',
 'Sleep time 0.9',
 'Sleep time 0.5',
 'Sleep time 0.4',
 'Sleep time 1.5',
 'Sleep time 1.3',
 'Sleep time 1.0',
 'Sleep time 0.3',
 'Sleep time 0.7',
 'Sleep time 0.6',
 'Sleep time 0.3',
 'Sleep time 0.8']

In [8]:
import time
import ray
import numpy as np
 
Stime = time.perf_counter()
 
@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)
 
@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)
 
@ray.remote
def sum_matrices(x, y):
    return np.add(x, y)
 
m1 = create_matrix.remote([1000, 1000])
m2 = create_matrix.remote([1000, 1000])
m3 = create_matrix.remote([1000, 1000])
m4 = create_matrix.remote([1000, 1000])
 
m12 = multiply_matrices.remote(m1, m2)
m34 = multiply_matrices.remote(m3, m4)
 
a12_34 =  sum_matrices.remote(m12, m34)
 
## 结果
MM = ray.get(a12_34)
 
 
print(f"Finished in {time.perf_counter()-Stime:.2f}")

Finished in 0.34
